In [2]:
# Mencionando as bibliotecas do Facebook

from facebook_business.adobjects.adaccount import AdAccount
from facebook_business.adobjects.adsinsights import AdsInsights
from facebook_business.api import FacebookAdsApi

# Importando a biblioteca Pandas para tratar e exportar os dados em planilha

import pandas as pd

In [3]:
# Definindo as chaves do App para processar as requisições da API

access_token = 'EAAFIZA49DnfIBAOxCQ588NP1faGej6I0Ia25kNUxVw4hCkrhZB0Q5Xv8QWQhqEz1xvhTTG7z5SqfvFIaLdkwCgUZC0gFvN5Mp7EFv9YOjw44g3SrXdb3csqRErZB1lSw7ojo0Sd7jMGFdvrEnqPS8GwOPdJEaZByGMGorlrvapZAzbwGGAgI9iqJ0pefmaKLwZD'
ad_account_id = 'act_539429070963780'
app_secret = 'f0dd5e05667d8855201a49ec5fdb2f82'
app_id = '361084599115250'
FacebookAdsApi.init(access_token=access_token)

In [4]:
# Escrevendo as informações que queremos puxar (consultar documentação da API)

fields = [
    'reach',
    'spend',
    'impressions',
    'frequency',
]
params = {
    'time_range': {'since':'2022-03-09','until':'2022-04-08'},
    'filtering': [],
    'level': 'campaign',
}

In [14]:
# Mostrando os resultados

AdAccount(ad_account_id).get_insights(
    fields=fields,
    params=params,
)

[]

In [10]:
# Definindo qual é o DataFrame que estamos trabalhando

df = pd.DataFrame({AdAccount(ad_account_id).get_insights(
    fields=fields,
    params=params,
)})

In [12]:
# Exportando para CSV

csv = df.to_csv(index=False)